In [182]:
# Importation des modules

import pandas as pd
from gensim.models import Word2Vec
import random
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import datetime

#Affichage de toutes les colonnes

pd.set_option('display.max_columns', 500)

In [183]:
w2v_model_all = Word2Vec.load('results/word2vec_all_300.model')

In [184]:
df0 = pd.read_csv("data/data_cleaned_NLP.csv", sep = ',', encoding = 'latin-1')

In [185]:
df = df0[['Date', 'Groupe', 'Orateur', 'tokenized_replique']].copy()

In [ ]:
df['days'] = df.apply(lambda row : (datetime.date(int(row.Date[:4]),int(row.Date[5:7]),int(row.Date[8:])) - 
                    datetime.date(2017,6,28)).days, axis = 1)

In [138]:
def add_groupe(L, etiquette):
    return [i + etiquette for i in L]

In [139]:
def calcule_WMD_intervalle(d1, d2):
    print('JOURS : ', d1, ' ', d2)
    df1 = df[df['days'] >= d1]
    df1 = df1[df1['days'] < d2]
    
    df_Novice = df1[df1['Groupe'] == 'Novice']
    df_Exp = df1[df1['Groupe'] == 'Exp']
    
    #display(df_Novice)
    try:
        df_Novice = df1[df1['Groupe'] == 'Novice'].sample(50).dropna()
        df_Exp = df1[df1['Groupe'] == 'Exp'].sample(50).dropna()
        print(df_Exp.shape, df_Novice.shape)
    except:
        return np.nan
    
    phrases_Exp = [df_Exp.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Exp.shape[0])]
    phrases_Novice = [df_Novice.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Novice.shape[0])]
    
    WMD = []
    

    for i in phrases_Novice:
        value = np.mean([w2v_model_all.wv.wmdistance(add_groupe(i, '_Novice'), 
                                                     add_groupe(j, '_Exp')) for j in phrases_Exp])
        print(value)
        if str(value) == 'inf':
            print(i)
        WMD.append(value)
    
    px.line([np.mean(WMD[:i]) for i in range(1, len(WMD))]).show()
    
    return np.mean(WMD)

In [140]:
#calcule_WMD_intervalle(322, 336)

In [141]:
#L = [calcule_WMD_intervalle(i, i + 14) for i in range(0, 1094, 14)]
#px.line(y = L, x = [i for i in range(0, 1094, 14)])

values = list((pd.read_csv('results/WMDinter_temporel.csv')['0']))
dates = list((pd.read_csv('results/WMDinter_temporel.csv')['Unnamed: 0']))

In [142]:
#pd.DataFrame(L, [i for i in range(0, 1094, 14)]).to_csv('results/WMDinter_temporel0.csv')

In [143]:
px.line(y = values, x = dates)

In [158]:
date_label = [datetime.timedelta(days=i) + datetime.date(2017,6,28) for i in range(14, 1094, 14)]

In [159]:
px.line(y = [np.mean(values[:i]) for i in range(1,len(values))], x = date_label, 
        labels = {'x' : 'Dates', 'y' : 'Moyenne cumulée des distances'})

On observe la moyenne cumulée des distances en fonction du temps. On constate une moyenne forte en début d'exercice, puis une relaxation pour atteindre une valeur limite.
On peut interpréter cela comme : 
- un début d'exercice où les novices et les expérimentés parlent différemment, la distance entre les deux langages est raltivement grande.
- une période d'apprentissage (du 20 septembre 2017 au 16 mai 2018) où les différences entre novices et expérimentés tendent à s'estomper (mais dans quel sens ? on s'attend à ce que les novices s'adaptent aux expérimentés, et pas forcement l'inverse), la distance diminue.
- une période de différenciation (du 16 mai 2018 au 12 décembre 2018), où la distance augmente légerement : comme si un des deux camps voulait se différencier de l'autre.
- une période stable (du 12 décembre 2018 à la fin de l'exercice).

## Analyse personnalisée député par député

In [175]:
# Députés Novices parlant le plus (classement par nombre d'intervention)

dict(df0[df0['Groupe'] == 'Novice'].Orateur.value_counts())

{'laurent pietraszewski': 3435,
 'francois ruffin': 3125,
 'sabine rubin': 2189,
 'adrien taquet': 1781,
 'roland lescure': 1705,
 'laetitia avia': 1637,
 'emilie chalas': 1579,
 'cendra motin': 1557,
 'yael braun pivet': 1286,
 'catherine fabre': 1159,
 'coralie dubost': 1138,
 'christophe naegelen': 1134,
 'thomas mesnier': 1121,
 'jean baptiste djebbari': 1096,
 'marie christine verdier jouclas': 986,
 'nadia hai': 986,
 'bruno studer': 895,
 'sacha houlie': 880,
 'gilles le gendre': 836,
 'berangere couillard': 714,
 'veronique riotton': 675,
 'benedicte peyrol': 603,
 'alexandra louis': 603,
 'raphael gauvain': 598,
 'raphael gerard': 594,
 'mohamed laqhila': 586,
 'agnes thill': 568,
 'martine wonner': 542,
 'cecile rilhac': 527,
 'stephanie rist': 478,
 'zivka park': 469,
 'perrine goulet': 461,
 'vincent thiebaut': 446,
 'jean francois mbaye': 439,
 'alice thourot': 391,
 'alexandre holroyd': 361,
 'caroline abadie': 336,
 'annie chapelier': 326,
 'fiona lazaar': 322,
 'damien 

In [177]:
orateurs_novices = list(dict(df0[df0['Groupe'] == 'Novice'].Orateur.value_counts()).keys())

In [181]:
df0

,Unnamed: 0,Unnamed: 0.1,Orateur,Date,Nature.de.séance,Président.de.séance,Sujet.débattu,Réplique,Didascalie,Président.de.séance_propre,nom.famille,sexe,age,groupe.sigle,commissions,nb.mandats,cabcollab,duree.pol,clustRFSP,clustVEP,hautdip,naissance_an,majo,profsigni2,ID,Groupe,time_floor,tokenized_replique,tokenized_didascalie,stemmed_replique,stemmed_didascalie
0,0,1,adrien morenas,2019-11-04,Projet de loi de finances pour 2020,Présidence de M. Hugues Renson,NaN,Si vous y donnez un avis favorable madame la ...,NaN,Présidence de M. Hugues Renson,Morenas,M,35,LREM,DvptDurable,0,False,0,1 - Soc. civile,6  Exp. Pol.,Doct,1982,True,NaN,1,Exp,2019-11-01,si donnez avis favorable madame secrétaire éta...,NaN,si don avis favor madam secrétair état dissip ...,NaN
1,1,2,adrien morenas,2019-11-04,Projet de loi de finances pour 2020,Présidence de M. Hugues Renson,NaN,Plus généralement je soutiendrai un autre ame...,NaN,Présidence de M. Hugues Renson,Morenas,M,35,LREM,DvptDurable,0,False,0,1 - Soc. civile,6  Exp. Pol.,Doct,1982,True,NaN,2,Exp,2019-11-01,plus généralement soutiendrai autre amendement...,NaN,plus général soutiendr autr amend vis demand g...,NaN
2,2,3,adrien morenas,2019-07-02,Questions au Gouvernement,Présidence de M. Richard Ferrand,Convention citoyenne sur le climat,Ma question sadresse à M. François de Rugy ...,NaN,Présidence de M. Richard Ferrand,Morenas,M,35,LREM,DvptDurable,0,False,0,1 - Soc. civile,6  Exp. Pol.,Doct,1982,True,NaN,3,Exp,2019-07-01,question adresse françois rugy ministre état m...,NaN,question adress françois rugy ministr état min...,NaN
3,3,4,adrien morenas,2018-02-01,Accès à leau,Présidence de Mme Carole Bureau-Bonnard,Discussion générale,Le 1,NaN,Présidence de Mme Carole Bureau-Bonnard,Morenas,M,35,LREM,DvptDurable,0,False,0,1 - Soc. civile,6  Exp. Pol.,Doct,1982,True,NaN,4,Exp,2018-02-01,NaN,NaN,NaN,NaN
4,4,5,adrien morenas,2020-05-08,Prorogation de létat durgence sanitaire,Présidence de Mme Laetitia Saint-Paul,Discussion des articles,Cette autorisation strictement encadrée par l...,NaN,Présidence de Mme Laetitia Saint-Paul,Morenas,M,35,LREM,DvptDurable,0,False,0,1 - Soc. civile,6  Exp. Pol.,Doct,1982,True,NaN,5,Exp,2020-05-01,cette autorisation strictement encadrée décret...,NaN,cet autoris strict encadr décret appliqu effet...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499838,499838,499839,zivka park,2018-11-05,Projet de loi de finances pour 2019,Présidence de M. Marc Le Fur,NaN,rapporteure pour avis de la commission du dév...,NaN,Présidence de M. Marc Le Fur,Park,F,32,LREM,DvptDurable,0,False,0,1 - Soc. civile,1 - Novice,ScPo,1985,True,cadsupprive,499839,Novice,2018-11-01,rapporteure avis commission développement dura...,NaN,rapporteur avis commiss développ durabl aménag...,NaN
499839,499839,499840,zivka park,2019-09-12,Mobilités,Présidence de M. Sylvain Waserman,Discussion des articles,rapporteure. Avis défavorable. Les agents des...,NaN,Présidence de M. Sylvain Waserman,Park,F,32,LREM,DvptDurable,0,False,0,1 - Soc. civile,1 - Novice,ScPo,1985,True,cadsupprive,499840,Novice,2019-09-01,rapporteure avis défavorable agents sociétés s...,NaN,rapporteur avis défavor agent societ sécur pri...,NaN
499840,499840,499841,zivka park,2018-02-20,Débat sur la couverture numérique du territoire,Présidence de M. Sylvain Waserman,NaN,Selon lARCEP les opérateurs dépenseront 3 à ...,NaN,Présidence de M. Sylvain Waserman,Park,F,32,LREM,DvptDurable,0,False,0,1 - Soc. civile,1 - Novice,ScPo,1985,True,cadsupprive,499841,Novice,2018-02-01,selon arcep opérateurs dépenseront milliards e...,NaN,selon arcep oper dépens milliard euros plus le...,NaN
499841,499841,499842,zivka park,2019-06-14,Mobilités,Présidence de Mme Carole Bureau-Bonnard,Discussion des articles,rapporteure. Je comprends votre préoccupation...,NaN,Présidence de Mme Carole Bureau-Bonnard,Park,F,32,LREM,DvptDurable,0,False,0,1 - Soc. civile,1 - Novice,ScPo,19

In [ ]:
def calcule_WMD_intervalle_personnalise(orateur, d1, d2):
    print('JOURS : ', d1, ' ', d2)
    df1 = df[df['days'] >= d1]
    df1 = df1[df1['days'] < d2]
    
    df_Novice = df1[df1['Orateur'] == orateur]
    df_Exp = df1[df1['Groupe'] == 'Exp']
    
    #display(df_Novice)
    try:
        df_Novice = df1[df1['Groupe'] == 'Novice'].dropna()
        df_Exp = df1[df1['Groupe'] == 'Exp'].sample(50).dropna()
        print(df_Exp.shape, df_Novice.shape)
    except:
        return np.nan
    
    phrases_Exp = [df_Exp.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Exp.shape[0])]
    phrases_Novice = [df_Novice.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Novice.shape[0])]
    
    WMD = []
    

    for i in phrases_Novice:
        value = np.mean([w2v_model_all.wv.wmdistance(add_groupe(i, '_Novice'), 
                                                     add_groupe(j, '_Exp')) for j in phrases_Exp])
        print(value)
        if str(value) == 'inf':
            print(i)
        WMD.append(value)
    
    px.line([np.mean(WMD[:i]) for i in range(1, len(WMD))]).show()
    
    return np.mean(WMD)

In [ ]:
calcule_WMD_intervalle_personnalise('laurent pietraszewski', 0, 14)